In [1]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
from sklearn import preprocessing
# Network Parameters
num_hidden_1 = 512 # 1st layer num features
num_hidden_2 = 256
num_hidden_3 = 128
num_hidden_4 = 64# 2nd layer num features (the latent dim)
num_input = 20
num_classes = 3

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [2]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [3]:
from numpy import genfromtxt
X_tr = genfromtxt('./NewDataForBiplab/DataRabi2017-2018/Train_2017-2018Rabi_NDVI.csv', delimiter=',')
X_te = genfromtxt('./NewDataForBiplab/DataRabi2017-2018/Test_2017-2018Rabi_NDVI.csv', delimiter=',')

In [4]:
X_tr = X_tr[1:,:]
X_te = X_te[1:,:]
print(X_tr.shape)
print(X_te.shape)

(240, 20)
(61, 20)


In [5]:
print(X_tr.max())
print(X_te.max())

119.250652688249
54.2852080730307


In [6]:
import numpy as np
X_tr = np.array(X_tr)
X_te = np.array(X_te)
print(X_tr.shape)

(240, 20)


In [7]:
X_tr = preprocessing.scale(X_tr)
X_te = preprocessing.scale(X_te)
print(X_tr.shape)

(240, 20)


In [8]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others', 'Index']
data1 = pandas.read_csv('./NewDataForBiplab/DataRabi2017-2018/Train_FieldDataRabi2017-2018.csv', names=colnames)
data2 = pandas.read_csv('./NewDataForBiplab/DataRabi2017-2018/Test_FieldDataRabi2017-2018.csv', names=colnames)

In [9]:
ctype1 = data1.CropType.tolist()
ctype2 = data2.CropType.tolist()
print(ctype1)

['CType', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'S

In [10]:
p = np.array(ctype1)
p1=  np.array(ctype2)
print(p.shape)
print(p1.shape)

(241,)
(62,)


In [11]:
lbl1 = ctype1
label = []
for i in range(1,241):
    if(lbl1[i] == "Wheat"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "Mustard"):
        label += [2]
    
print(label)
y_tr=np.array(label)

lbl1 = ctype2
label = []
for i in range(1,62):
    if(lbl1[i] == "Wheat"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "Mustard"):
        label += [2]
    
print(label)
y_te=np.array(label)
print()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]



In [12]:
x1=[]
x2=[]
x3=[]
l1=[]
l2=[]
l3=[]
for i in range(0, y_te.shape[0]):
    if(y_te[i]==0):
        x1.append(X_te[i])
        l1.append([1,0,0])
    if(y_te[i]==1):
        x2.append(X_te[i])
        l2.append([0,1,0])
    if(y_te[i]==2):
        x3.append(X_te[i])
        l3.append([0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)

print(l2.shape)

(29, 3)


In [13]:
from sklearn.utils import shuffle
X_tr, y_tr = shuffle(X_tr, y_tr, random_state = 0)
X_te, y_te = shuffle(X_te, y_te, random_state = 0)

In [14]:
print(X_tr.shape)
print(X_te.shape)
print(y_tr.shape)
print(y_te.shape)

(240, 20)
(61, 20)
(240,)
(61,)


In [15]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_te)
y_train = np_utils.to_categorical(y_tr)
print(y_train.shape)
print(y_test.shape)

(240, 3)
(61, 3)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 16
num_batch = 27

# Training
for i in range(0,10000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_tr[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += batch_size

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.46989867
Epoch: 2 cost = 0.40355304
Epoch: 3 cost = 0.35451686
Epoch: 4 cost = 0.31677434
Epoch: 5 cost = 0.28790565
Epoch: 6 cost = 0.26583490
Epoch: 7 cost = 0.24863954
Epoch: 8 cost = 0.23450348
Epoch: 9 cost = 0.22229528
Epoch: 10 cost = 0.21167083
Epoch: 11 cost = 0.20262269
Epoch: 12 cost = 0.19496669
Epoch: 13 cost = 0.18839481
Epoch: 14 cost = 0.18263538
Epoch: 15 cost = 0.17747358
Epoch: 16 cost = 0.17276872
Epoch: 17 cost = 0.16844554
Epoch: 18 cost = 0.16446260
Epoch: 19 cost = 0.16078511
Epoch: 20 cost = 0.15737639
Epoch: 21 cost = 0.15420336
Epoch: 22 cost = 0.15124109
Epoch: 23 cost = 0.14847103
Epoch: 24 cost = 0.14587729
Epoch: 25 cost = 0.14344306
Epoch: 26 cost = 0.14115080
Epoch: 27 cost = 0.13898444
Epoch: 28 cost = 0.13693039
Epoch: 29 cost = 0.13497765
Epoch: 30 cost = 0.13311704
Epoch: 31 cost = 0.13134110
Epoch: 32 cost = 0.12964460
Epoch: 33 cost = 0.12802530
Epoch: 34 cost = 0.12648427
Epoch: 35 cost = 0.12502407
Epoch: 36 cost = 0.12364546
E

Epoch: 295 cost = 0.07236652
Epoch: 296 cost = 0.07233238
Epoch: 297 cost = 0.07229872
Epoch: 298 cost = 0.07226548
Epoch: 299 cost = 0.07223260
Epoch: 300 cost = 0.07220005
Epoch: 301 cost = 0.07216780
Epoch: 302 cost = 0.07213580
Epoch: 303 cost = 0.07210405
Epoch: 304 cost = 0.07207252
Epoch: 305 cost = 0.07204122
Epoch: 306 cost = 0.07201014
Epoch: 307 cost = 0.07197928
Epoch: 308 cost = 0.07194866
Epoch: 309 cost = 0.07191827
Epoch: 310 cost = 0.07188815
Epoch: 311 cost = 0.07185828
Epoch: 312 cost = 0.07182869
Epoch: 313 cost = 0.07179938
Epoch: 314 cost = 0.07177038
Epoch: 315 cost = 0.07174170
Epoch: 316 cost = 0.07171332
Epoch: 317 cost = 0.07168527
Epoch: 318 cost = 0.07165755
Epoch: 319 cost = 0.07163016
Epoch: 320 cost = 0.07160309
Epoch: 321 cost = 0.07157635
Epoch: 322 cost = 0.07154992
Epoch: 323 cost = 0.07152381
Epoch: 324 cost = 0.07149799
Epoch: 325 cost = 0.07147247
Epoch: 326 cost = 0.07144724
Epoch: 327 cost = 0.07142228
Epoch: 328 cost = 0.07139756
Epoch: 329 cos

Epoch: 591 cost = 0.06635895
Epoch: 592 cost = 0.06633689
Epoch: 593 cost = 0.06631524
Epoch: 594 cost = 0.06629416
Epoch: 595 cost = 0.06627374
Epoch: 596 cost = 0.06625407
Epoch: 597 cost = 0.06623514
Epoch: 598 cost = 0.06621695
Epoch: 599 cost = 0.06619947
Epoch: 600 cost = 0.06618266
Epoch: 601 cost = 0.06616644
Epoch: 602 cost = 0.06615075
Epoch: 603 cost = 0.06613555
Epoch: 604 cost = 0.06612075
Epoch: 605 cost = 0.06610632
Epoch: 606 cost = 0.06609217
Epoch: 607 cost = 0.06607829
Epoch: 608 cost = 0.06606461
Epoch: 609 cost = 0.06605111
Epoch: 610 cost = 0.06603773
Epoch: 611 cost = 0.06602449
Epoch: 612 cost = 0.06601131
Epoch: 613 cost = 0.06599821
Epoch: 614 cost = 0.06598517
Epoch: 615 cost = 0.06597217
Epoch: 616 cost = 0.06595920
Epoch: 617 cost = 0.06594628
Epoch: 618 cost = 0.06593339
Epoch: 619 cost = 0.06592054
Epoch: 620 cost = 0.06590774
Epoch: 621 cost = 0.06589500
Epoch: 622 cost = 0.06588231
Epoch: 623 cost = 0.06586970
Epoch: 624 cost = 0.06585717
Epoch: 625 cos

Epoch: 889 cost = 0.06371627
Epoch: 890 cost = 0.06370985
Epoch: 891 cost = 0.06370346
Epoch: 892 cost = 0.06369710
Epoch: 893 cost = 0.06369077
Epoch: 894 cost = 0.06368448
Epoch: 895 cost = 0.06367821
Epoch: 896 cost = 0.06367198
Epoch: 897 cost = 0.06366576
Epoch: 898 cost = 0.06365957
Epoch: 899 cost = 0.06365341
Epoch: 900 cost = 0.06364727
Epoch: 901 cost = 0.06364116
Epoch: 902 cost = 0.06363505
Epoch: 903 cost = 0.06362896
Epoch: 904 cost = 0.06362290
Epoch: 905 cost = 0.06361685
Epoch: 906 cost = 0.06361081
Epoch: 907 cost = 0.06360478
Epoch: 908 cost = 0.06359877
Epoch: 909 cost = 0.06359276
Epoch: 910 cost = 0.06358675
Epoch: 911 cost = 0.06358075
Epoch: 912 cost = 0.06357477
Epoch: 913 cost = 0.06356878
Epoch: 914 cost = 0.06356277
Epoch: 915 cost = 0.06355678
Epoch: 916 cost = 0.06355078
Epoch: 917 cost = 0.06354478
Epoch: 918 cost = 0.06353878
Epoch: 919 cost = 0.06353275
Epoch: 920 cost = 0.06352673
Epoch: 921 cost = 0.06352069
Epoch: 922 cost = 0.06351463
Epoch: 923 cos

Epoch: 1182 cost = 0.05831292
Epoch: 1183 cost = 0.05829289
Epoch: 1184 cost = 0.05827315
Epoch: 1185 cost = 0.05825370
Epoch: 1186 cost = 0.05823454
Epoch: 1187 cost = 0.05821567
Epoch: 1188 cost = 0.05819711
Epoch: 1189 cost = 0.05817887
Epoch: 1190 cost = 0.05816098
Epoch: 1191 cost = 0.05814345
Epoch: 1192 cost = 0.05812630
Epoch: 1193 cost = 0.05810957
Epoch: 1194 cost = 0.05809327
Epoch: 1195 cost = 0.05807741
Epoch: 1196 cost = 0.05806203
Epoch: 1197 cost = 0.05804713
Epoch: 1198 cost = 0.05803272
Epoch: 1199 cost = 0.05801881
Epoch: 1200 cost = 0.05800539
Epoch: 1201 cost = 0.05799246
Epoch: 1202 cost = 0.05798000
Epoch: 1203 cost = 0.05796800
Epoch: 1204 cost = 0.05795645
Epoch: 1205 cost = 0.05794532
Epoch: 1206 cost = 0.05793459
Epoch: 1207 cost = 0.05792425
Epoch: 1208 cost = 0.05791426
Epoch: 1209 cost = 0.05790461
Epoch: 1210 cost = 0.05789528
Epoch: 1211 cost = 0.05788625
Epoch: 1212 cost = 0.05787749
Epoch: 1213 cost = 0.05786900
Epoch: 1214 cost = 0.05786076
Epoch: 121

...

Epoch: 8508 cost = 0.05538958
Epoch: 8509 cost = 0.05538953
Epoch: 8510 cost = 0.05538948
Epoch: 8511 cost = 0.05538943
Epoch: 8512 cost = 0.05538938
Epoch: 8513 cost = 0.05538933
Epoch: 8514 cost = 0.05538928
Epoch: 8515 cost = 0.05538924
Epoch: 8516 cost = 0.05538918
Epoch: 8517 cost = 0.05538913
Epoch: 8518 cost = 0.05538909
Epoch: 8519 cost = 0.05538904
Epoch: 8520 cost = 0.05538899
Epoch: 8521 cost = 0.05538894
Epoch: 8522 cost = 0.05538889
Epoch: 8523 cost = 0.05538884
Epoch: 8524 cost = 0.05538879
Epoch: 8525 cost = 0.05538874
Epoch: 8526 cost = 0.05538870
Epoch: 8527 cost = 0.05538864
Epoch: 8528 cost = 0.05538859
Epoch: 8529 cost = 0.05538855
Epoch: 8530 cost = 0.05538850
Epoch: 8531 cost = 0.05538844
Epoch: 8532 cost = 0.05538840
Epoch: 8533 cost = 0.05538834
Epoch: 8534 cost = 0.05538829
Epoch: 8535 cost = 0.05538825
Epoch: 8536 cost = 0.05538820
Epoch: 8537 cost = 0.05538814
Epoch: 8538 cost = 0.05538810
Epoch: 8539 cost = 0.05538805
Epoch: 8540 cost = 0.05538800
Epoch: 854

Epoch: 8795 cost = 0.05537181
Epoch: 8796 cost = 0.05537173
Epoch: 8797 cost = 0.05537165
Epoch: 8798 cost = 0.05537156
Epoch: 8799 cost = 0.05537148
Epoch: 8800 cost = 0.05537140
Epoch: 8801 cost = 0.05537132
Epoch: 8802 cost = 0.05537123
Epoch: 8803 cost = 0.05537115
Epoch: 8804 cost = 0.05537107
Epoch: 8805 cost = 0.05537098
Epoch: 8806 cost = 0.05537090
Epoch: 8807 cost = 0.05537082
Epoch: 8808 cost = 0.05537073
Epoch: 8809 cost = 0.05537066
Epoch: 8810 cost = 0.05537057
Epoch: 8811 cost = 0.05537049
Epoch: 8812 cost = 0.05537040
Epoch: 8813 cost = 0.05537032
Epoch: 8814 cost = 0.05537024
Epoch: 8815 cost = 0.05537015
Epoch: 8816 cost = 0.05537006
Epoch: 8817 cost = 0.05536998
Epoch: 8818 cost = 0.05536990
Epoch: 8819 cost = 0.05536982
Epoch: 8820 cost = 0.05536973
Epoch: 8821 cost = 0.05536964
Epoch: 8822 cost = 0.05536956
Epoch: 8823 cost = 0.05536947
Epoch: 8824 cost = 0.05536939
Epoch: 8825 cost = 0.05536931
Epoch: 8826 cost = 0.05536922
Epoch: 8827 cost = 0.05536913
Epoch: 882

Epoch: 9102 cost = 0.05534902
Epoch: 9103 cost = 0.05534897
Epoch: 9104 cost = 0.05534892
Epoch: 9105 cost = 0.05534887
Epoch: 9106 cost = 0.05534881
Epoch: 9107 cost = 0.05534877
Epoch: 9108 cost = 0.05534871
Epoch: 9109 cost = 0.05534866
Epoch: 9110 cost = 0.05534861
Epoch: 9111 cost = 0.05534856
Epoch: 9112 cost = 0.05534851
Epoch: 9113 cost = 0.05534846
Epoch: 9114 cost = 0.05534841
Epoch: 9115 cost = 0.05534836
Epoch: 9116 cost = 0.05534831
Epoch: 9117 cost = 0.05534826
Epoch: 9118 cost = 0.05534821
Epoch: 9119 cost = 0.05534816
Epoch: 9120 cost = 0.05534811
Epoch: 9121 cost = 0.05534806
Epoch: 9122 cost = 0.05534801
Epoch: 9123 cost = 0.05534796
Epoch: 9124 cost = 0.05534791
Epoch: 9125 cost = 0.05534786
Epoch: 9126 cost = 0.05534781
Epoch: 9127 cost = 0.05534776
Epoch: 9128 cost = 0.05534771
Epoch: 9129 cost = 0.05534766
Epoch: 9130 cost = 0.05534761
Epoch: 9131 cost = 0.05534757
Epoch: 9132 cost = 0.05534752
Epoch: 9133 cost = 0.05534747
Epoch: 9134 cost = 0.05534743
Epoch: 913

Epoch: 9385 cost = 0.05533765
Epoch: 9386 cost = 0.05533761
Epoch: 9387 cost = 0.05533758
Epoch: 9388 cost = 0.05533755
Epoch: 9389 cost = 0.05533751
Epoch: 9390 cost = 0.05533748
Epoch: 9391 cost = 0.05533745
Epoch: 9392 cost = 0.05533741
Epoch: 9393 cost = 0.05533739
Epoch: 9394 cost = 0.05533736
Epoch: 9395 cost = 0.05533732
Epoch: 9396 cost = 0.05533729
Epoch: 9397 cost = 0.05533725
Epoch: 9398 cost = 0.05533722
Epoch: 9399 cost = 0.05533719
Epoch: 9400 cost = 0.05533716
Epoch: 9401 cost = 0.05533713
Epoch: 9402 cost = 0.05533709
Epoch: 9403 cost = 0.05533706
Epoch: 9404 cost = 0.05533703
Epoch: 9405 cost = 0.05533700
Epoch: 9406 cost = 0.05533697
Epoch: 9407 cost = 0.05533694
Epoch: 9408 cost = 0.05533690
Epoch: 9409 cost = 0.05533687
Epoch: 9410 cost = 0.05533684
Epoch: 9411 cost = 0.05533681
Epoch: 9412 cost = 0.05533678
Epoch: 9413 cost = 0.05533675
Epoch: 9414 cost = 0.05533671
Epoch: 9415 cost = 0.05533668
Epoch: 9416 cost = 0.05533665
Epoch: 9417 cost = 0.05533662
Epoch: 941

Epoch: 9697 cost = 0.05532872
Epoch: 9698 cost = 0.05532869
Epoch: 9699 cost = 0.05532867
Epoch: 9700 cost = 0.05532864
Epoch: 9701 cost = 0.05532861
Epoch: 9702 cost = 0.05532859
Epoch: 9703 cost = 0.05532857
Epoch: 9704 cost = 0.05532854
Epoch: 9705 cost = 0.05532851
Epoch: 9706 cost = 0.05532848
Epoch: 9707 cost = 0.05532846
Epoch: 9708 cost = 0.05532843
Epoch: 9709 cost = 0.05532841
Epoch: 9710 cost = 0.05532839
Epoch: 9711 cost = 0.05532836
Epoch: 9712 cost = 0.05532833
Epoch: 9713 cost = 0.05532831
Epoch: 9714 cost = 0.05532828
Epoch: 9715 cost = 0.05532826
Epoch: 9716 cost = 0.05532823
Epoch: 9717 cost = 0.05532821
Epoch: 9718 cost = 0.05532818
Epoch: 9719 cost = 0.05532816
Epoch: 9720 cost = 0.05532813
Epoch: 9721 cost = 0.05532811
Epoch: 9722 cost = 0.05532808
Epoch: 9723 cost = 0.05532806
Epoch: 9724 cost = 0.05532803
Epoch: 9725 cost = 0.05532800
Epoch: 9726 cost = 0.05532798
Epoch: 9727 cost = 0.05532795
Epoch: 9728 cost = 0.05532793
Epoch: 9729 cost = 0.05532790
Epoch: 973

In [22]:
batch_size = 16
num_batch = 27

# Training
for i in range(0,10000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_tr[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += batch_size

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.05532144
Epoch: 2 cost = 0.05532142
Epoch: 3 cost = 0.05532140
Epoch: 4 cost = 0.05532138
Epoch: 5 cost = 0.05532135
Epoch: 6 cost = 0.05532133
Epoch: 7 cost = 0.05532130
Epoch: 8 cost = 0.05532128
Epoch: 9 cost = 0.05532126
Epoch: 10 cost = 0.05532124
Epoch: 11 cost = 0.05532122
Epoch: 12 cost = 0.05532119
Epoch: 13 cost = 0.05532117
Epoch: 14 cost = 0.05532115
Epoch: 15 cost = 0.05532113
Epoch: 16 cost = 0.05532110
Epoch: 17 cost = 0.05532108
Epoch: 18 cost = 0.05532106
Epoch: 19 cost = 0.05532104
Epoch: 20 cost = 0.05532102
Epoch: 21 cost = 0.05532099
Epoch: 22 cost = 0.05532097
Epoch: 23 cost = 0.05532095
Epoch: 24 cost = 0.05532092
Epoch: 25 cost = 0.05532090
Epoch: 26 cost = 0.05532087
Epoch: 27 cost = 0.05532086
Epoch: 28 cost = 0.05532083
Epoch: 29 cost = 0.05532081
Epoch: 30 cost = 0.05532079
Epoch: 31 cost = 0.05532077
Epoch: 32 cost = 0.05532075
Epoch: 33 cost = 0.05532072
Epoch: 34 cost = 0.05532070
Epoch: 35 cost = 0.05532068
Epoch: 36 cost = 0.05532066
E

Epoch: 318 cost = 0.05531464
Epoch: 319 cost = 0.05531462
Epoch: 320 cost = 0.05531460
Epoch: 321 cost = 0.05531458
Epoch: 322 cost = 0.05531456
Epoch: 323 cost = 0.05531454
Epoch: 324 cost = 0.05531452
Epoch: 325 cost = 0.05531449
Epoch: 326 cost = 0.05531448
Epoch: 327 cost = 0.05531445
Epoch: 328 cost = 0.05531443
Epoch: 329 cost = 0.05531441
Epoch: 330 cost = 0.05531440
Epoch: 331 cost = 0.05531437
Epoch: 332 cost = 0.05531435
Epoch: 333 cost = 0.05531433
Epoch: 334 cost = 0.05531431
Epoch: 335 cost = 0.05531429
Epoch: 336 cost = 0.05531427
Epoch: 337 cost = 0.05531425
Epoch: 338 cost = 0.05531423
Epoch: 339 cost = 0.05531421
Epoch: 340 cost = 0.05531419
Epoch: 341 cost = 0.05531417
Epoch: 342 cost = 0.05531415
Epoch: 343 cost = 0.05531413
Epoch: 344 cost = 0.05531411
Epoch: 345 cost = 0.05531409
Epoch: 346 cost = 0.05531407
Epoch: 347 cost = 0.05531404
Epoch: 348 cost = 0.05531402
Epoch: 349 cost = 0.05531401
Epoch: 350 cost = 0.05531398
Epoch: 351 cost = 0.05531396
Epoch: 352 cos

Epoch: 629 cost = 0.05530848
Epoch: 630 cost = 0.05530846
Epoch: 631 cost = 0.05530844
Epoch: 632 cost = 0.05530842
Epoch: 633 cost = 0.05530840
Epoch: 634 cost = 0.05530838
Epoch: 635 cost = 0.05530836
Epoch: 636 cost = 0.05530834
Epoch: 637 cost = 0.05530833
Epoch: 638 cost = 0.05530830
Epoch: 639 cost = 0.05530829
Epoch: 640 cost = 0.05530826
Epoch: 641 cost = 0.05530825
Epoch: 642 cost = 0.05530823
Epoch: 643 cost = 0.05530821
Epoch: 644 cost = 0.05530819
Epoch: 645 cost = 0.05530817
Epoch: 646 cost = 0.05530815
Epoch: 647 cost = 0.05530813
Epoch: 648 cost = 0.05530811
Epoch: 649 cost = 0.05530809
Epoch: 650 cost = 0.05530808
Epoch: 651 cost = 0.05530805
Epoch: 652 cost = 0.05530804
Epoch: 653 cost = 0.05530802
Epoch: 654 cost = 0.05530800
Epoch: 655 cost = 0.05530798
Epoch: 656 cost = 0.05530796
Epoch: 657 cost = 0.05530794
Epoch: 658 cost = 0.05530792
Epoch: 659 cost = 0.05530790
Epoch: 660 cost = 0.05530789
Epoch: 661 cost = 0.05530786
Epoch: 662 cost = 0.05530785
Epoch: 663 cos

Epoch: 921 cost = 0.05530304
Epoch: 922 cost = 0.05530302
Epoch: 923 cost = 0.05530301
Epoch: 924 cost = 0.05530298
Epoch: 925 cost = 0.05530296
Epoch: 926 cost = 0.05530295
Epoch: 927 cost = 0.05530293
Epoch: 928 cost = 0.05530291
Epoch: 929 cost = 0.05530290
Epoch: 930 cost = 0.05530288
Epoch: 931 cost = 0.05530286
Epoch: 932 cost = 0.05530284
Epoch: 933 cost = 0.05530283
Epoch: 934 cost = 0.05530281
Epoch: 935 cost = 0.05530279
Epoch: 936 cost = 0.05530277
Epoch: 937 cost = 0.05530275
Epoch: 938 cost = 0.05530273
Epoch: 939 cost = 0.05530271
Epoch: 940 cost = 0.05530270
Epoch: 941 cost = 0.05530268
Epoch: 942 cost = 0.05530266
Epoch: 943 cost = 0.05530264
Epoch: 944 cost = 0.05530262
Epoch: 945 cost = 0.05530260
Epoch: 946 cost = 0.05530259
Epoch: 947 cost = 0.05530256
Epoch: 948 cost = 0.05530255
Epoch: 949 cost = 0.05530254
Epoch: 950 cost = 0.05530252
Epoch: 951 cost = 0.05530250
Epoch: 952 cost = 0.05530248
Epoch: 953 cost = 0.05530246
Epoch: 954 cost = 0.05530244
Epoch: 955 cos

Epoch: 1224 cost = 0.05529768
Epoch: 1225 cost = 0.05529766
Epoch: 1226 cost = 0.05529764
Epoch: 1227 cost = 0.05529763
Epoch: 1228 cost = 0.05529761
Epoch: 1229 cost = 0.05529759
Epoch: 1230 cost = 0.05529758
Epoch: 1231 cost = 0.05529756
Epoch: 1232 cost = 0.05529754
Epoch: 1233 cost = 0.05529752
Epoch: 1234 cost = 0.05529750
Epoch: 1235 cost = 0.05529749
Epoch: 1236 cost = 0.05529747
Epoch: 1237 cost = 0.05529745
Epoch: 1238 cost = 0.05529743
Epoch: 1239 cost = 0.05529742
Epoch: 1240 cost = 0.05529740
Epoch: 1241 cost = 0.05529738
Epoch: 1242 cost = 0.05529737
Epoch: 1243 cost = 0.05529735
Epoch: 1244 cost = 0.05529733
Epoch: 1245 cost = 0.05529732
Epoch: 1246 cost = 0.05529730
Epoch: 1247 cost = 0.05529728
Epoch: 1248 cost = 0.05529726
Epoch: 1249 cost = 0.05529725
Epoch: 1250 cost = 0.05529723
Epoch: 1251 cost = 0.05529721
Epoch: 1252 cost = 0.05529719
Epoch: 1253 cost = 0.05529718
Epoch: 1254 cost = 0.05529716
Epoch: 1255 cost = 0.05529714
Epoch: 1256 cost = 0.05529713
Epoch: 125

...

Epoch: 8824 cost = 0.05505966
Epoch: 8825 cost = 0.05505963
Epoch: 8826 cost = 0.05505961
Epoch: 8827 cost = 0.05505958
Epoch: 8828 cost = 0.05505955
Epoch: 8829 cost = 0.05505953
Epoch: 8830 cost = 0.05505950
Epoch: 8831 cost = 0.05505948
Epoch: 8832 cost = 0.05505945
Epoch: 8833 cost = 0.05505943
Epoch: 8834 cost = 0.05505940
Epoch: 8835 cost = 0.05505938
Epoch: 8836 cost = 0.05505934
Epoch: 8837 cost = 0.05505932
Epoch: 8838 cost = 0.05505929
Epoch: 8839 cost = 0.05505926
Epoch: 8840 cost = 0.05505923
Epoch: 8841 cost = 0.05505921
Epoch: 8842 cost = 0.05505918
Epoch: 8843 cost = 0.05505915
Epoch: 8844 cost = 0.05505913
Epoch: 8845 cost = 0.05505910
Epoch: 8846 cost = 0.05505907
Epoch: 8847 cost = 0.05505904
Epoch: 8848 cost = 0.05505902
Epoch: 8849 cost = 0.05505899
Epoch: 8850 cost = 0.05505896
Epoch: 8851 cost = 0.05505893
Epoch: 8852 cost = 0.05505890
Epoch: 8853 cost = 0.05505887
Epoch: 8854 cost = 0.05505884
Epoch: 8855 cost = 0.05505881
Epoch: 8856 cost = 0.05505879
Epoch: 885

Epoch: 9120 cost = 0.05504202
Epoch: 9121 cost = 0.05504190
Epoch: 9122 cost = 0.05504179
Epoch: 9123 cost = 0.05504166
Epoch: 9124 cost = 0.05504154
Epoch: 9125 cost = 0.05504143
Epoch: 9126 cost = 0.05504130
Epoch: 9127 cost = 0.05504118
Epoch: 9128 cost = 0.05504106
Epoch: 9129 cost = 0.05504094
Epoch: 9130 cost = 0.05504082
Epoch: 9131 cost = 0.05504069
Epoch: 9132 cost = 0.05504057
Epoch: 9133 cost = 0.05504045
Epoch: 9134 cost = 0.05504032
Epoch: 9135 cost = 0.05504020
Epoch: 9136 cost = 0.05504008
Epoch: 9137 cost = 0.05503995
Epoch: 9138 cost = 0.05503983
Epoch: 9139 cost = 0.05503970
Epoch: 9140 cost = 0.05503958
Epoch: 9141 cost = 0.05503947
Epoch: 9142 cost = 0.05503934
Epoch: 9143 cost = 0.05503921
Epoch: 9144 cost = 0.05503909
Epoch: 9145 cost = 0.05503896
Epoch: 9146 cost = 0.05503884
Epoch: 9147 cost = 0.05503871
Epoch: 9148 cost = 0.05503859
Epoch: 9149 cost = 0.05503847
Epoch: 9150 cost = 0.05503834
Epoch: 9151 cost = 0.05503822
Epoch: 9152 cost = 0.05503809
Epoch: 915

Epoch: 9396 cost = 0.05502107
Epoch: 9397 cost = 0.05502104
Epoch: 9398 cost = 0.05502101
Epoch: 9399 cost = 0.05502098
Epoch: 9400 cost = 0.05502095
Epoch: 9401 cost = 0.05502093
Epoch: 9402 cost = 0.05502090
Epoch: 9403 cost = 0.05502087
Epoch: 9404 cost = 0.05502084
Epoch: 9405 cost = 0.05502081
Epoch: 9406 cost = 0.05502078
Epoch: 9407 cost = 0.05502076
Epoch: 9408 cost = 0.05502073
Epoch: 9409 cost = 0.05502070
Epoch: 9410 cost = 0.05502067
Epoch: 9411 cost = 0.05502065
Epoch: 9412 cost = 0.05502062
Epoch: 9413 cost = 0.05502059
Epoch: 9414 cost = 0.05502056
Epoch: 9415 cost = 0.05502053
Epoch: 9416 cost = 0.05502051
Epoch: 9417 cost = 0.05502048
Epoch: 9418 cost = 0.05502046
Epoch: 9419 cost = 0.05502043
Epoch: 9420 cost = 0.05502041
Epoch: 9421 cost = 0.05502037
Epoch: 9422 cost = 0.05502035
Epoch: 9423 cost = 0.05502033
Epoch: 9424 cost = 0.05502030
Epoch: 9425 cost = 0.05502027
Epoch: 9426 cost = 0.05502025
Epoch: 9427 cost = 0.05502022
Epoch: 9428 cost = 0.05502020
Epoch: 942

Epoch: 9674 cost = 0.05501570
Epoch: 9675 cost = 0.05501568
Epoch: 9676 cost = 0.05501566
Epoch: 9677 cost = 0.05501565
Epoch: 9678 cost = 0.05501564
Epoch: 9679 cost = 0.05501562
Epoch: 9680 cost = 0.05501561
Epoch: 9681 cost = 0.05501559
Epoch: 9682 cost = 0.05501558
Epoch: 9683 cost = 0.05501557
Epoch: 9684 cost = 0.05501556
Epoch: 9685 cost = 0.05501554
Epoch: 9686 cost = 0.05501553
Epoch: 9687 cost = 0.05501552
Epoch: 9688 cost = 0.05501550
Epoch: 9689 cost = 0.05501549
Epoch: 9690 cost = 0.05501547
Epoch: 9691 cost = 0.05501546
Epoch: 9692 cost = 0.05501544
Epoch: 9693 cost = 0.05501543
Epoch: 9694 cost = 0.05501542
Epoch: 9695 cost = 0.05501541
Epoch: 9696 cost = 0.05501539
Epoch: 9697 cost = 0.05501538
Epoch: 9698 cost = 0.05501537
Epoch: 9699 cost = 0.05501535
Epoch: 9700 cost = 0.05501534
Epoch: 9701 cost = 0.05501532
Epoch: 9702 cost = 0.05501531
Epoch: 9703 cost = 0.05501530
Epoch: 9704 cost = 0.05501529
Epoch: 9705 cost = 0.05501527
Epoch: 9706 cost = 0.05501526
Epoch: 970

Epoch: 9973 cost = 0.05501216
Epoch: 9974 cost = 0.05501215
Epoch: 9975 cost = 0.05501214
Epoch: 9976 cost = 0.05501213
Epoch: 9977 cost = 0.05501212
Epoch: 9978 cost = 0.05501211
Epoch: 9979 cost = 0.05501210
Epoch: 9980 cost = 0.05501209
Epoch: 9981 cost = 0.05501208
Epoch: 9982 cost = 0.05501206
Epoch: 9983 cost = 0.05501206
Epoch: 9984 cost = 0.05501205
Epoch: 9985 cost = 0.05501204
Epoch: 9986 cost = 0.05501203
Epoch: 9987 cost = 0.05501201
Epoch: 9988 cost = 0.05501200
Epoch: 9989 cost = 0.05501199
Epoch: 9990 cost = 0.05501198
Epoch: 9991 cost = 0.05501197
Epoch: 9992 cost = 0.05501196
Epoch: 9993 cost = 0.05501195
Epoch: 9994 cost = 0.05501195
Epoch: 9995 cost = 0.05501193
Epoch: 9996 cost = 0.05501193
Epoch: 9997 cost = 0.05501191
Epoch: 9998 cost = 0.05501190
Epoch: 9999 cost = 0.05501189
Epoch: 10000 cost = 0.05501188


In [23]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
print(y_test.shape)

(61, 3)


In [25]:
# on 1000 epochcura
print(sess.run(accuracy, feed_dict={X: X_te, Y: y_test}))

0.88524586


In [20]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
print(res1)
print(res2)
print(res3)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1]
[1 0 2 2 2 1 2 2]


In [21]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
confusion.append([x1,x2,x3])

confusion=np.array(confusion)
print(confusion)

[[22  2  0]
 [ 1 28  0]
 [ 1  2  5]]


In [45]:
#SVM Classifier
from sklearn.svm import SVC  
from sklearn.metrics import confusion_matrix, cohen_kappa_score
svclassifier = SVC(kernel='rbf')  
svclassifier.fit(X_tr, y_tr)  

y_pred = svclassifier.predict(X_te)  

In [46]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, y_pred)
print(accuracy)

0.9180327868852459


In [47]:
print(confusion_matrix(y_te,y_pred)) 

[[22  2  0]
 [ 1 28  0]
 [ 1  1  6]]


In [48]:
print(cohen_kappa_score(y_pred, y_te))

0.8612374886260237
